In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.spatial.distance as ssd
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
from scipy import stats
pd.options.mode.chained_assignment = None 

In [48]:
#read file as pandas dataframe
data = pd.read_csv('variant_surveillance.tsv', sep = '\t',dtype = {'Variant': 'string','Is reference?': 'string'})
data = data[data['Clade'].notna()]
data.head()

/tmp/ipykernel_154078/797462131.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('variant_surveillance.tsv', sep = '\t',dtype = {'Variant': 'string','Is reference?': 'string'})


,Accession ID,Type,Clade,Pango lineage,Pangolin version,AA Substitutions,Variant,Collection date,Location,Host,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content,Sequence length
0,EPI_ISL_10010236,betacoronavirus,GRA,BA.1.17,PANGO-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,NaN,0.379635,29747
1,EPI_ISL_10010227,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.009446,0.379794,29747
2,EPI_ISL_10010232,betacoronavirus,GRA,BA.1.17,PLEARN-v1.8,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-25,Oceania / Australia / Australian Capital Terri...,Human,2022-02-17,<NA>,True,NaN,NaN,0.000134,0.379585,29747
3,EPI_ISL_3578085,betacoronavirus,GK,B.1.617.2,PLEARN-v1.8,"(N_D377Y,M_I82T,Spike_D950N,N_R203M,NSP12_G671...",VOC Delta GK (B.1.617.2+AY.*) first detected i...,2021-08-18,Europe / Turkey,Human,2021-08-23,<NA>,True,True,NaN,0.000101,0.379853,29754
4,EPI_ISL_10299251,betacoronavirus,GRA,BA.1,PLEARN-v1.8,"(NSP5_P132H,Spike_T95I,Spike_A67V,Spike_K417N,...",VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,2022-01-17,Europe / Germany / North Rhine-Westphalia,Human,2022-02-24,<NA>,True,NaN,NaN,0.048908,0.380371,29729


In [49]:
#clean dataset and change collection date to datetime object
data = data[['Clade', 'Location', 'Collection date']]
data['Collection date'] =  pd.to_datetime(data['Collection date'])    #.dt.strftime('%Y/%m')
time_frame = (data['Collection date'] >= '2020/01/01')
data = data.loc[time_frame]
data.head()

,Clade,Location,Collection date
0,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25
1,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25
2,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25
3,GK,Europe / Turkey,2021-08-18
4,GRA,Europe / Germany / North Rhine-Westphalia,2022-01-17


In [50]:
data['Date']= data['Collection date'].dt.to_period('W')
data.head()

,Clade,Location,Collection date,Date
0,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
1,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
2,GRA,Oceania / Australia / Australian Capital Terri...,2022-01-25,2022-01-24/2022-01-30
3,GK,Europe / Turkey,2021-08-18,2021-08-16/2021-08-22
4,GRA,Europe / Germany / North Rhine-Westphalia,2022-01-17,2022-01-17/2022-01-23


In [51]:
df = data.copy()

In [52]:
#specify strings for each continent
df.loc[df['Location'].str.contains('Brazil'), 'Location'] = 'Brazil'

In [53]:
df1 = df[df['Location'] == 'Brazil']
df1.head()

,Clade,Location,Collection date,Date
28128,O,Brazil,2021-10-24,2021-10-18/2021-10-24
36319,GK,Brazil,2021-08-04,2021-08-02/2021-08-08
37344,GK,Brazil,2021-08-04,2021-08-02/2021-08-08
37347,GK,Brazil,2021-08-04,2021-08-02/2021-08-08
42336,GR,Brazil,2020-10-15,2020-10-12/2020-10-18


In [54]:
df1.shape

(149791, 4)

In [55]:
#create cumulative sum of clade submission counts per month
df1 = df1.groupby(['Clade','Date','Location']).size().unstack(fill_value = 0)
# print(df1)
df1 = df1.reset_index()
df1.head()

Location,Clade,Date,Brazil
0,G,2019-12-30/2020-01-05,1
1,G,2020-02-24/2020-03-01,4
2,G,2020-03-02/2020-03-08,4
3,G,2020-03-09/2020-03-15,17
4,G,2020-03-16/2020-03-22,51


In [56]:
df1.shape

(508, 3)

In [57]:
df_global = df1.copy()
df_slide = df1.copy()

In [58]:
df_global = df_global.groupby('Clade').cumsum()
# print(df_global)
#add columns back
df_global['Date'] = df1['Date']
# print(df_global)
df_global['Clade'] = df1['Clade']
df_global[df_global['Clade'] == 'GRA']

Location,Brazil,Date,Clade
311,1,2020-12-28/2021-01-03,GRA
312,2,2021-06-07/2021-06-13,GRA
313,3,2021-07-12/2021-07-18,GRA
314,4,2021-08-23/2021-08-29,GRA
315,10,2021-11-22/2021-11-28,GRA
316,19,2021-11-29/2021-12-05,GRA
317,108,2021-12-06/2021-12-12,GRA
318,387,2021-12-13/2021-12-19,GRA
319,1068,2021-12-20/2021-12-26,GRA
320,3643,2021-12-27/2022-01-02,GRA


In [59]:
df_slide = df_slide[df_slide['Clade'] != 'GKA']
df_slide = df_slide[df_slide['Clade'] != 'GV']
df_slide = df_slide[df_slide['Clade'] != 'O']
df_slide = df_slide[df_slide['Clade'] != 'G']
df_slide = df_slide[df_slide['Clade'] != 'S']
df_slide = df_slide[df_slide['Clade'] != 'L']
df_slide = df_slide[df_slide['Clade'] != 'V']
df_slide.head()

Location,Clade,Date,Brazil
106,GH,2020-03-02/2020-03-08,3
107,GH,2020-03-09/2020-03-15,6
108,GH,2020-03-16/2020-03-22,11
109,GH,2020-03-23/2020-03-29,11
110,GH,2020-03-30/2020-04-05,4


In [60]:
df_slide.shape

(271, 3)

In [61]:
df_BR_qrt = df_slide[['Brazil', 'Clade', 'Date']]
df_BR_qrt.rename(columns = {'Brazil':'Values'}, inplace = True)
df_BR_qrt.head()

Location,Values,Clade,Date
106,3,GH,2020-03-02/2020-03-08
107,6,GH,2020-03-09/2020-03-15
108,11,GH,2020-03-16/2020-03-22
109,11,GH,2020-03-23/2020-03-29
110,4,GH,2020-03-30/2020-04-05


In [62]:
df_BR_qrt.shape

(271, 3)

In [63]:
def transform(dataframe):
    dataframe = dataframe.pivot(index = 'Date', columns = 'Clade', values = 'Values')
    dataframe = dataframe.replace([0, np.inf, np.nan], 1)
    dataframe = dataframe.reset_index()
    return dataframe
#     print(dataframe.head())

In [64]:
df_BR_qrt = transform(df_BR_qrt)
df_BR_qrt

Clade,Date,GH,GK,GR,GRA,GRY
0,2019-12-30/2020-01-05,1.0,1.0,9.0,1.0,1.0
1,2020-02-24/2020-03-01,1.0,1.0,4.0,1.0,1.0
2,2020-03-02/2020-03-08,3.0,1.0,9.0,1.0,1.0
3,2020-03-09/2020-03-15,6.0,1.0,23.0,1.0,1.0
4,2020-03-16/2020-03-22,11.0,1.0,124.0,1.0,1.0
5,2020-03-23/2020-03-29,11.0,1.0,155.0,1.0,1.0
6,2020-03-30/2020-04-05,4.0,1.0,190.0,1.0,1.0
7,2020-04-06/2020-04-12,3.0,1.0,198.0,1.0,1.0
8,2020-04-13/2020-04-19,3.0,1.0,255.0,1.0,1.0
9,2020-04-20/2020-04-26,7.0,1.0,280.0,1.0,1.0


In [65]:
df_BR_qrt['range'] = df_BR_qrt['Date'].astype(str)
df_BR_qrt

Clade,Date,GH,GK,GR,GRA,GRY,range
0,2019-12-30/2020-01-05,1.0,1.0,9.0,1.0,1.0,2019-12-30/2020-01-05
1,2020-02-24/2020-03-01,1.0,1.0,4.0,1.0,1.0,2020-02-24/2020-03-01
2,2020-03-02/2020-03-08,3.0,1.0,9.0,1.0,1.0,2020-03-02/2020-03-08
3,2020-03-09/2020-03-15,6.0,1.0,23.0,1.0,1.0,2020-03-09/2020-03-15
4,2020-03-16/2020-03-22,11.0,1.0,124.0,1.0,1.0,2020-03-16/2020-03-22
5,2020-03-23/2020-03-29,11.0,1.0,155.0,1.0,1.0,2020-03-23/2020-03-29
6,2020-03-30/2020-04-05,4.0,1.0,190.0,1.0,1.0,2020-03-30/2020-04-05
7,2020-04-06/2020-04-12,3.0,1.0,198.0,1.0,1.0,2020-04-06/2020-04-12
8,2020-04-13/2020-04-19,3.0,1.0,255.0,1.0,1.0,2020-04-13/2020-04-19
9,2020-04-20/2020-04-26,7.0,1.0,280.0,1.0,1.0,2020-04-20/2020-04-26


In [66]:
tmp = df_BR_qrt['range'].str.split(pat = '/', expand = True)
tmp.head()

,0,1
0,2019-12-30,2020-01-05
1,2020-02-24,2020-03-01
2,2020-03-02,2020-03-08
3,2020-03-09,2020-03-15
4,2020-03-16,2020-03-22


In [67]:
tmp.columns = ['Start','end']
tmp.head()

,Start,end
0,2019-12-30,2020-01-05
1,2020-02-24,2020-03-01
2,2020-03-02,2020-03-08
3,2020-03-09,2020-03-15
4,2020-03-16,2020-03-22


In [68]:
df_BR_qrt['start'] = tmp['Start']
df_BR_qrt['end'] = tmp['end']
df_BR_qrt.head()

Clade,Date,GH,GK,GR,GRA,GRY,range,start,end
0,2019-12-30/2020-01-05,1.0,1.0,9.0,1.0,1.0,2019-12-30/2020-01-05,2019-12-30,2020-01-05
1,2020-02-24/2020-03-01,1.0,1.0,4.0,1.0,1.0,2020-02-24/2020-03-01,2020-02-24,2020-03-01
2,2020-03-02/2020-03-08,3.0,1.0,9.0,1.0,1.0,2020-03-02/2020-03-08,2020-03-02,2020-03-08
3,2020-03-09/2020-03-15,6.0,1.0,23.0,1.0,1.0,2020-03-09/2020-03-15,2020-03-09,2020-03-15
4,2020-03-16/2020-03-22,11.0,1.0,124.0,1.0,1.0,2020-03-16/2020-03-22,2020-03-16,2020-03-22


In [69]:
df_BR_qrt.shape

(120, 9)

In [70]:
#clades contains all GISAID clades, VOC contains GISAID clades containing VOCs
#change clade input to whichever you want to analyze
clades = ['G','GH','GK','GR','GRA','GRY','GV','L','O','S','V']
VOC = ['GH','GK','GR','GRA','GRY']

clade_input = VOC    ####

#function to generate distance matrix based on quarter, year, and continent
def create_distance_matrix(df,quarter_continent):
    tmp = df['Clade'].str.split('vs', expand=True)
    clade_list = tmp[0].unique()
    distance_matrix = pd.DataFrame(index = clade_list)
    distance_matrix[clade_list] = np.nan
    #fill in distance matrix
    for i in clade_list:
        for j in clade_list:
            if i == j:
                distance_matrix.loc[i,j] = 0
            else:
                distance_matrix.loc[i,j] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j),
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
                distance_matrix.loc[j,i] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
    return distance_matrix

#function to generate difference matrix based on quarter, year, and continent
def create_difference_matrix(df,quarter_continent):
    tmp = df['Clade'].str.split('vs', expand=True)
    clade_list = tmp[0].unique()
    difference_matrix = pd.DataFrame(index = clade_list)
    difference_matrix[clade_list] = np.nan
    #fill in difference matrix
    for i in clade_list:
        for j in clade_list:
            if i == j:
                difference_matrix.loc[i,j] = 0
            else:
                difference_matrix.loc[i,j] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j),
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
                difference_matrix.loc[j,i] = (abs((df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                        quarter_continent].iloc[0] + 
                                            df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j), 
                                                   quarter_continent].iloc[0])/2))
    #if row_vs_col is a negative value, make it negative in difference matrix
    for i,v in difference_matrix.iterrows():
        for j in difference_matrix:
            if i != j:
                if (df.loc[df['Clade'] == '{a}vs{b}'.format(a=i,b=j),quarter_continent].iloc[0]) < 0:
                    difference_matrix.loc[i,j] *= -1
    return difference_matrix

#function to calculate the log transformed ratio of two clades
def transform_ratio(clade_1, clade_2):
    return np.log10(clade_1/clade_2)

def time_frame(df,time_window):
    dataframe = df.loc[df['Date'].isin(time_window)]
    dataframe.iloc[:,2:] = dataframe.iloc[:,2:].cumsum()
    for i in dataframe.columns.tolist():
        if dataframe[i].isnull().all():
            dataframe = dataframe.drop(i, axis=1)
    return dataframe

def time_frame_no_cumsum(df,time_window):
    dataframe = df.loc[df['Date'].isin(time_window)]
    return dataframe

#function returns slope of clade advantage plots
def slope_calc(day, clades):
    par = np.polyfit(day, clades, 1, full = True)
    slope= par[0][0]
    return slope

def clade_comparisons(df):
    clade_list = [i for i in clade_input if i in df.columns.tolist()]
    df['date_ordinal'] = pd.to_datetime(df['start']).apply(lambda date: date.toordinal())
    df['days'] = df['date_ordinal'] - 737425
    #calculate log ratio for each clade comparison
    for i in clade_list:
        for j in clade_list:
            if i != j:
                df['{a}vs{b}'.format(a=i,b=j)] = df.apply(lambda row : 
                                                                        transform_ratio(row['{a}'.format(a=i)],
                                                                                        row['{b}'.format(b=j)]),
                                                                                        axis = 1)
    clade_list.append('date_ordinal')
    df = df.drop(clade_list, axis=1)
    return df

In [71]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [72]:
df_BR_qrt = clade_comparisons(df_BR_qrt)
df_BR_qrt

Clade,Date,range,start,end,days,GHvsGK,GHvsGR,GHvsGRA,GHvsGRY,GKvsGH,GKvsGR,GKvsGRA,GKvsGRY,GRvsGH,GRvsGK,GRvsGRA,GRvsGRY,GRAvsGH,GRAvsGK,GRAvsGR,GRAvsGRY,GRYvsGH,GRYvsGK,GRYvsGR,GRYvsGRA
0,2019-12-30/2020-01-05,2019-12-30/2020-01-05,2019-12-30,2020-01-05,-2,0.000000,-0.954243,0.000000,0.000000,0.000000,-0.954243,0.000000,0.000000,0.954243,0.954243,0.954243,0.954243,0.000000,0.000000,-0.954243,0.000000,0.000000,0.000000,-0.954243,0.000000
1,2020-02-24/2020-03-01,2020-02-24/2020-03-01,2020-02-24,2020-03-01,54,0.000000,-0.602060,0.000000,0.000000,0.000000,-0.602060,0.000000,0.000000,0.602060,0.602060,0.602060,0.602060,0.000000,0.000000,-0.602060,0.000000,0.000000,0.000000,-0.602060,0.000000
2,2020-03-02/2020-03-08,2020-03-02/2020-03-08,2020-03-02,2020-03-08,61,0.477121,-0.477121,0.477121,0.477121,-0.477121,-0.954243,0.000000,0.000000,0.477121,0.954243,0.954243,0.954243,-0.477121,0.000000,-0.954243,0.000000,-0.477121,0.000000,-0.954243,0.000000
3,2020-03-09/2020-03-15,2020-03-09/2020-03-15,2020-03-09,2020-03-15,68,0.778151,-0.583577,0.778151,0.778151,-0.778151,-1.361728,0.000000,0.000000,0.583577,1.361728,1.361728,1.361728,-0.778151,0.000000,-1.361728,0.000000,-0.778151,0.000000,-1.361728,0.000000
4,2020-03-16/2020-03-22,2020-03-16/2020-03-22,2020-03-16,2020-03-22,75,1.041393,-1.052029,1.041393,1.041393,-1.041393,-2.093422,0.000000,0.000000,1.052029,2.093422,2.093422,2.093422,-1.041393,0.000000,-2.093422,0.000000,-1.041393,0.000000,-2.093422,0.000000
5,2020-03-23/2020-03-29,2020-03-23/2020-03-29,2020-03-23,2020-03-29,82,1.041393,-1.148939,1.041393,1.041393,-1.041393,-2.190332,0.000000,0.000000,1.148939,2.190332,2.190332,2.190332,-1.041393,0.000000,-2.190332,0.000000,-1.041393,0.000000,-2.190332,0.000000
6,2020-03-30/2020-04-05,2020-03-30/2020-04-05,2020-03-30,2020-04-05,89,0.602060,-1.676694,0.602060,0.602060,-0.602060,-2.278754,0.000000,0.000000,1.676694,2.278754,2.278754,2.278754,-0.602060,0.000000,-2.278754,0.000000,-0.602060,0.000000,-2.278754,0.000000
7,2020-04-06/2020-04-12,2020-04-06/2020-04-12,2020-04-06,2020-04-12,96,0.477121,-1.819544,0.477121,0.477121,-0.477121,-2.296665,0.000000,0.000000,1.819544,2.296665,2.296665,2.296665,-0.477121,0.000000,-2.296665,0.000000,-0.477121,0.000000,-2.296665,0.000000
8,2020-04-13/2020-04-19,2020-04-13/2020-04-19,2020-04-13,2020-04-19,103,0.477121,-1.929419,0.477121,0.477121,-0.477121,-2.406540,0.000000,0.000000,1.929419,2.406540,2.406540,2.406540,-0.477121,0.000000,-2.406540,0.000000,-0.477121,0.000000,-2.406540,0.000000
9,2020-04-20/2020-04-26,2020-04-20/2020-04-26,2020-04-20,2020-04-26,110,0.845098,-1.602060,0.845098,0.845098,-0.845098,-2.447158,0.000000,0.000000,1.602060,2.447158,2.447158,2.447158,-0.845098,0.000000,-2.447158,0.000000,-0.845098,0.000000,-2.447158,0.000000


In [73]:
df_BR_qrt.shape

(120, 25)

In [74]:
m1_2020_BR = []
m2_2020_BR = []
m3_2020_BR = []
m4_2020_BR = []
m5_2020_BR = []
m6_2020_BR = []
m7_2020_BR = []
m8_2020_BR = []

In [75]:
for i in range(20):
    m1_2020_BR.append(slope_calc(df_BR_qrt.iloc[2:7,4],df_BR_qrt.iloc[2:7,i + 5]))
    m2_2020_BR.append(slope_calc(df_BR_qrt.iloc[7:12,4],df_BR_qrt.iloc[7:12,i + 5]))
    m3_2020_BR.append(slope_calc(df_BR_qrt.iloc[12:17,4],df_BR_qrt.iloc[12:17,i + 5]))
    m4_2020_BR.append(slope_calc(df_BR_qrt.iloc[17:22,4],df_BR_qrt.iloc[17:22,i + 5]))
    m5_2020_BR.append(slope_calc(df_BR_qrt.iloc[22:27,4],df_BR_qrt.iloc[22:27,i + 5]))
    m6_2020_BR.append(slope_calc(df_BR_qrt.iloc[27:32,4],df_BR_qrt.iloc[27:32,i + 5]))
    m7_2020_BR.append(slope_calc(df_BR_qrt.iloc[32:37,4],df_BR_qrt.iloc[32:37,i + 5]))
    m8_2020_BR.append(slope_calc(df_BR_qrt.iloc[37:42,4],df_BR_qrt.iloc[37:42,i + 5]))

In [76]:
monthly_slopes = pd.DataFrame()

monthly_slopes['m1_2020_BR'] = m1_2020_BR
monthly_slopes['m2_2020_BR'] = m2_2020_BR
monthly_slopes['m3_2020_BR'] = m3_2020_BR
monthly_slopes['m4_2020_BR'] = m4_2020_BR
monthly_slopes['m5_2020_BR'] = m5_2020_BR
monthly_slopes['m6_2020_BR'] = m6_2020_BR
monthly_slopes['m7_2020_BR'] = m7_2020_BR
monthly_slopes['m8_2020_BR'] = m8_2020_BR

monthly_slopes.index = df_BR_qrt.columns[5:]

In [77]:
monthly_slopes['Clade'] = monthly_slopes.index

In [78]:
monthly_slopes

,m1_2020_BR,m2_2020_BR,m3_2020_BR,m4_2020_BR,m5_2020_BR,m6_2020_BR,m7_2020_BR,m8_2020_BR,Clade
Clade,,,,,,,,,
GHvsGK,0.00733,-0.005031,-0.012901,0.000000,0.017578,0.000000,0.000000,0.004300,GHvsGK
GHvsGR,-0.04235,0.002720,-0.014913,0.000831,0.018026,-0.001375,-0.003651,-0.006487,GHvsGR
GHvsGRA,0.00733,-0.005031,-0.012901,0.000000,0.017578,0.000000,0.000000,0.004300,GHvsGRA
GHvsGRY,0.00733,-0.005031,-0.012901,0.000000,0.017578,0.000000,0.000000,0.004300,GHvsGRY
GKvsGH,-0.00733,0.005031,0.012901,0.000000,-0.017578,0.000000,0.000000,-0.004300,GKvsGH
GKvsGR,-0.04968,0.007752,-0.002011,0.000831,0.000448,-0.001375,-0.003651,-0.010787,GKvsGR
GKvsGRA,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,GKvsGRA
GKvsGRY,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,GKvsGRY
GRvsGH,0.04235,-0.002720,0.014913,-0.000831,-0.018026,0.001375,0.003651,0.006487,GRvsGH


In [79]:
monthly_slopes

,m1_2020_BR,m2_2020_BR,m3_2020_BR,m4_2020_BR,m5_2020_BR,m6_2020_BR,m7_2020_BR,m8_2020_BR,Clade
Clade,,,,,,,,,
GHvsGK,0.00733,-0.005031,-0.012901,0.000000,0.017578,0.000000,0.000000,0.004300,GHvsGK
GHvsGR,-0.04235,0.002720,-0.014913,0.000831,0.018026,-0.001375,-0.003651,-0.006487,GHvsGR
GHvsGRA,0.00733,-0.005031,-0.012901,0.000000,0.017578,0.000000,0.000000,0.004300,GHvsGRA
GHvsGRY,0.00733,-0.005031,-0.012901,0.000000,0.017578,0.000000,0.000000,0.004300,GHvsGRY
GKvsGH,-0.00733,0.005031,0.012901,0.000000,-0.017578,0.000000,0.000000,-0.004300,GKvsGH
GKvsGR,-0.04968,0.007752,-0.002011,0.000831,0.000448,-0.001375,-0.003651,-0.010787,GKvsGR
GKvsGRA,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,GKvsGRA
GKvsGRY,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,GKvsGRY
GRvsGH,0.04235,-0.002720,0.014913,-0.000831,-0.018026,0.001375,0.003651,0.006487,GRvsGH


In [80]:
monthly_slopes = monthly_slopes.melt(id_vars=['Clade'],
       var_name='Location',
       value_name='Slope')

In [81]:
monthly_slopes[['ID1','ID2']] = monthly_slopes.Clade.str.split('vs',expand=True)
monthly_slopes.head()

,Clade,Location,Slope,ID1,ID2
0,GHvsGK,m1_2020_BR,0.00733,GH,GK
1,GHvsGR,m1_2020_BR,-0.04235,GH,GR
2,GHvsGRA,m1_2020_BR,0.00733,GH,GRA
3,GHvsGRY,m1_2020_BR,0.00733,GH,GRY
4,GKvsGH,m1_2020_BR,-0.00733,GK,GH


In [82]:
monthly_slopes = monthly_slopes.drop(['Clade'], axis=1)
monthly_slopes.head()

,Location,Slope,ID1,ID2
0,m1_2020_BR,0.00733,GH,GK
1,m1_2020_BR,-0.04235,GH,GR
2,m1_2020_BR,0.00733,GH,GRA
3,m1_2020_BR,0.00733,GH,GRY
4,m1_2020_BR,-0.00733,GK,GH


In [83]:
monthly_slopes['Month_Year'] = monthly_slopes.Location.str.split('_',2)
monthly_slopes['Location'] = monthly_slopes.Month_Year.apply(lambda x: x[-1])
monthly_slopes['Month_Year'] = monthly_slopes.Month_Year.apply(lambda x: x[:2])
monthly_slopes.head()

,Location,Slope,ID1,ID2,Month_Year
0,BR,0.00733,GH,GK,"[m1, 2020]"
1,BR,-0.04235,GH,GR,"[m1, 2020]"
2,BR,0.00733,GH,GRA,"[m1, 2020]"
3,BR,0.00733,GH,GRY,"[m1, 2020]"
4,BR,-0.00733,GK,GH,"[m1, 2020]"


In [84]:
monthly_slopes['Month_Year'] = monthly_slopes['Month_Year'].str.join('_')
monthly_slopes.head()

,Location,Slope,ID1,ID2,Month_Year
0,BR,0.00733,GH,GK,m1_2020
1,BR,-0.04235,GH,GR,m1_2020
2,BR,0.00733,GH,GRA,m1_2020
3,BR,0.00733,GH,GRY,m1_2020
4,BR,-0.00733,GK,GH,m1_2020


In [85]:
monthly_slopes = monthly_slopes[['ID1','ID2','Slope','Location','Month_Year']]
monthly_slopes.head()

,ID1,ID2,Slope,Location,Month_Year
0,GH,GK,0.00733,BR,m1_2020
1,GH,GR,-0.04235,BR,m1_2020
2,GH,GRA,0.00733,BR,m1_2020
3,GH,GRY,0.00733,BR,m1_2020
4,GK,GH,-0.00733,BR,m1_2020


In [86]:
monthly_slopes[['Month','Year']] = monthly_slopes['Month_Year'].str.split('_', expand=True)
monthly_slopes.head()

,ID1,ID2,Slope,Location,Month_Year,Month,Year
0,GH,GK,0.00733,BR,m1_2020,m1,2020
1,GH,GR,-0.04235,BR,m1_2020,m1,2020
2,GH,GRA,0.00733,BR,m1_2020,m1,2020
3,GH,GRY,0.00733,BR,m1_2020,m1,2020
4,GK,GH,-0.00733,BR,m1_2020,m1,2020


In [88]:
monthly_slopes = monthly_slopes.drop(['Month_Year'], axis=1)
monthly_slopes.head()

,ID1,ID2,Slope,Location,Month,Year
0,GH,GK,0.00733,BR,m1,2020
1,GH,GR,-0.04235,BR,m1,2020
2,GH,GRA,0.00733,BR,m1,2020
3,GH,GRY,0.00733,BR,m1,2020
4,GK,GH,-0.00733,BR,m1,2020
